In [7]:
%load_ext autoreload
%autoreload 2

# import all the necessary modules
from pathlib import Path
import random

from nerfstudio.models.nerfacto import NerfactoModelConfig
from nerfstudio.configs.base_config import ViewerConfig
from nerfstudio.configs.experiment_config import ExperimentConfig
from nerfstudio.pipelines.nesf_pipeline import NesfPipelineConfig
from nerfstudio.data.datamanagers.nesf_datamanager import NesfDataManagerConfig
from nerfstudio.data.dataparsers.nesf_dataparser import NesfDataParserConfig
from nerfstudio.cameras.camera_optimizers import CameraOptimizerConfig
from nerfstudio.engine.optimizers import AdamOptimizerConfig, RAdamOptimizerConfig
from nerfstudio.engine.trainer import TrainerConfig
import torch

from utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of nerfstudio.pipelines.nesf_pipeline failed: Traceback (most recent call last):
  File "/data/vision/polina/projects/wmh/dhollidt/conda/envs/nerfstudio/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 261, in check
    superreload(m, reload, self.old_objects)
  File "/data/vision/polina/projects/wmh/dhollidt/conda/envs/nerfstudio/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 459, in superreload
    module = reload(module)
  File "/data/vision/polina/projects/wmh/dhollidt/conda/envs/nerfstudio/lib/python3.8/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 604, in _exec
  File "<frozen importlib._bootstrap_external>", line 843, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/YOUR/PATH/HERE/nerfstudio_fork/nerfstudio/pipelines/nesf_pipeline.py", line 33, in <module>
    class NesfPipelineConfig(cfg.Instantia

In [8]:
data_config_path = Path("/YOUR/PATH/HERE/playground/nesf_test_config.json")

OUTPUT_DIR = Path("/YOUR/PATH/HERE/playground/own_loading/")
DATA_PATH = Path("/data/vision/polina/scratch/clintonw/datasets/kubric/klevr/0")


trainConfig = TrainerConfig(
    method_name="nerfacto",
    experiment_name="/tmp",
    data=DATA_PATH,
    output_dir=OUTPUT_DIR,
    steps_per_eval_batch=500,
    steps_per_save=2000,
    max_num_iterations=30000,
    mixed_precision=True,
    pipeline=NesfPipelineConfig(
        datamanager=NesfDataManagerConfig(
            dataparser=NesfDataParserConfig(data_config=data_config_path),
            train_num_rays_per_batch=4096,
            eval_num_rays_per_batch=4096,
            camera_optimizer=CameraOptimizerConfig(
                mode="off", optimizer=AdamOptimizerConfig(lr=6e-4, eps=1e-8, weight_decay=1e-2)
            ),
        ),
        model=NerfactoModelConfig(eval_num_rays_per_chunk=1 << 15),
    ),
    optimizers={
        "proposal_networks": {
            "optimizer": AdamOptimizerConfig(lr=1e-2, eps=1e-15),
            "scheduler": None,
        },
        "fields": {
            "optimizer": AdamOptimizerConfig(lr=1e-2, eps=1e-15),
            "scheduler": None,
        },
    },
    viewer=ViewerConfig(num_rays_per_chunk=1 << 15),
    vis="tensorboard"
)


trainConfig.set_timestamp()
trainConfig.pipeline.datamanager.dataparser.data = trainConfig.data
trainConfig.save_config()

trainer = trainConfig.setup(local_rank=0, world_size=1)
trainer.setup()

[12:32:19] Saving config to:                                                                    ]8;id=896854;file:///YOUR/PATH/HERE/nerfstudio_fork/nerfstudio/configs/experiment_config.py\experiment_config.py]8;;\:]8;id=46811;file:///YOUR/PATH/HERE/nerfstudio_fork/nerfstudio/configs/experiment_config.py#124\124]8;;\
           /YOUR/PATH/HERE/playground/own_loading/tmp/                         
           nerfacto/2023-02-05_123219/config.yml                                                                        

[12:32:19] Saving checkpoints to:                                                                         ]8;id=829350;file:///YOUR/PATH/HERE/nerfstudio_fork/nerfstudio/engine/trainer.py\trainer.py]8;;\:]8;id=103638;file:///YOUR/PATH/HERE/nerfstudio_fork/nerfstudio/engine/trainer.py#120\120]8;;\
           /YOUR/PATH/HERE/playground/own_loading/tmp/nerfacto/2               
           023-02-05_123219/nerfstudio_models                                                                           

logging events to: 
/YOUR/PATH/HERE/playground/own_loading/tmp/nerfacto/2023-02-05_123219

TypeError: __init__() missing 2 required positional arguments: 'config' and 'device'